In [ ]:
#uploading files from google colab
from google.colab import files
fil=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
#Loading Data from kaggle
!pip install -q kaggle
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c dogs-vs-cats

100% 542M/543M [00:03<00:00, 187MB/s]
100% 543M/543M [00:03<00:00, 176MB/s]
 98% 265M/271M [00:01<00:00, 187MB/s]
100% 271M/271M [00:01<00:00, 174MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 78.1MB/s]


In [ ]:
#Unzip train and test1 zip files and remove zip files
!unzip train.zip
!rm train.zip
!unzip test1.zip 
!rm test1.zip

Streaming output truncated to the last 5000 lines.
  inflating: test1/5499.jpg          
  inflating: test1/55.jpg            
  inflating: test1/550.jpg           
  inflating: test1/5500.jpg          
  inflating: test1/5501.jpg          
  inflating: test1/5502.jpg          
  inflating: test1/5503.jpg          
  inflating: test1/5504.jpg          
  inflating: test1/5505.jpg          
  inflating: test1/5506.jpg          
  inflating: test1/5507.jpg          
  inflating: test1/5508.jpg          
  inflating: test1/5509.jpg          
  inflating: test1/551.jpg           
  inflating: test1/5510.jpg          
  inflating: test1/5511.jpg          
  inflating: test1/5512.jpg          
  inflating: test1/5513.jpg          
  inflating: test1/5514.jpg          
  inflating: test1/5515.jpg          
  inflating: test1/5516.jpg          
  inflating: test1/5517.jpg          
  inflating: test1/5518.jpg          
  inflating: test1/5519.jpg          
  inflating: test1/552.jpg           

In [ ]:
#Import files
import os
from torch.utils.data import Dataset,ConcatDataset,DataLoader
from torchvision import transforms
from PIL import Image
import torchvision
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch
import copy
import time
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
import torch.utils.data as data

#Train and test directories
train_dir='/content/train'
test_dir='/content/test1'
train_files=os.listdir(train_dir)
test_files=os.listdir(test_dir)

#Train test splittting data
train_list, val_list = train_test_split(train_files, test_size=0.2)

#Mean and standard deviation defining
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)


In [ ]:
class CatDogDataset(Dataset):
  def __init__(self,file_list,dir,mode='train',transform=None):
    self.file_list=file_list
    self.dir=dir
    self.transform=transform
    self.mode=mode

    # Setting labels for cat and dog classes
    if self.mode == 'train':
      if 'dog' in self.file_list[0]:
        self.label=1
      else:
        self.label=0

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self,idx):
    img=Image.open(os.path.join(self.dir,self.file_list[idx]))
    if self.transform:
      img=self.transform(img)
      
    if self.mode == 'train':
      img=img.numpy()
      return img.astype('float32'),self.label
    else:
      img=img.numpy()
      return img.astype('float32'),self.file_list[idx]



In [ ]:
class Data_load:
  
  #Data Augumentation
  def transformation(self):
    data_transform=transforms.Compose([transforms.Resize(256),transforms.ColorJitter(),transforms.RandomCrop(224),transforms.RandomHorizontalFlip(),transforms.Resize(256),transforms.ToTensor(),transforms.Normalize(mean, std)])
    return data_transform

  def data_set(self,train_files):
    # Setting of files of train and validation
    cat_train=[tf for tf in train_list if 'cat' in tf]
    dog_train=[tf for tf in train_list if 'dog' in tf]
    cat_val=[tf for tf in val_list if 'cat' in tf]
    dog_val=[tf for tf in val_list if 'dog' in tf]

    data_transform=self.transformation()

    # Making of Training and Validation Datasets
    cats_train=CatDogDataset(cat_train,train_dir,transform=data_transform)
    dogs_train=CatDogDataset(dog_train,train_dir,transform=data_transform)
    catdogs_train=ConcatDataset([cats_train,dogs_train])
    cats_val=CatDogDataset(cat_val,train_dir,transform=data_transform)
    dogs_val=CatDogDataset(dog_val,train_dir,transform=data_transform)
    catdogs_val=ConcatDataset([cats_val,dogs_val])

    return catdogs_train,catdogs_val


  def data_load(self,train_files):
    catdogs_train,catdogs_val=self.data_set(train_files)
    batch_size=32
    #Loading Data into DataLoader with batch_size and shuffle

    train_dl = data.DataLoader(catdogs_train, batch_size=batch_size, shuffle=True)
    valid_dl = data.DataLoader(catdogs_val, batch_size=batch_size, shuffle=False)
    
    return train_dl,valid_dl

d1=Data_load()
train_dl,valid_dl=d1.data_load(train_files)



In [ ]:
 def freeze_weights():
    params_to_update = []

    #Loading model and Freezing of parameters
    model=torchvision.models.vgg16(pretrained=True)
    for param in model.parameters():
       param.requires_grad = False
       
    # Changes to the model

    model.classifier[6] = nn.Linear(4096, 2)
    for name,param in model.named_parameters():
          if param.requires_grad == True:
              params_to_update.append(param)
              print("\t",name) 
    return model,params_to_update

model,params_to_update=freeze_weights()




	 classifier.6.weight
	 classifier.6.bias


In [ ]:
device='cuda'
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params_to_update,lr=0.01)
dataloader_dict = {'train': train_dl, 'val': valid_dl}

In [ ]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):
    
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    model = model.to(device)
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):

              #Move inputs and dataloader to Cuda
                inputs = inputs.to(device)
                labels = labels.to(device)

                #Reinitializing gradients to 0
                optimizer.zero_grad()
                
                # Setting the gradient updation to true
                with torch.set_grad_enabled(phase == 'train'):
                    #Prediction of output
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #Calculation of loss
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        #Finding small derivatives with respect to all parameters
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            #Calculating Loss and Accuracy for each epoch

            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
              #Loading best weights into model on best accuracy acquired
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #Load weights into the model
  
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
num_epoch = 10
model = train_model(model, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/10
--------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/625 [00:00<?, ?it/s]

train Loss: 0.2867 Acc: 0.9694


  0%|          | 0/157 [00:00<?, ?it/s]

...

val Loss: 0.3380 Acc: 0.9836
Epoch 10/10
--------------------


  0%|          | 0/625 [00:00<?, ?it/s]

train Loss: 0.5372 Acc: 0.9785


  0%|          | 0/157 [00:00<?, ?it/s]

val Loss: 0.3595 Acc: 0.9834
Training complete in 105m 37s
Best val Acc: 0.986200


In [ ]:
   #Save weights of model
   torch.save(model.state_dict(),'model_10.pt') 